In [27]:
import os

import geopandas as gpd
import fiona

import rasterio

import matplotlib.pyplot as plt
import numpy as np

In [2]:
in_labels = r"/media/ross/ssd/00_2015_DAR_marinedebris/maui/labels/final_marine_debris_labels_MA_utm4n.gpkg"
in_tile_index = r"/media/ross/ssd/00_2015_DAR_marinedebris/maui/tiles_shp/maui_gdal_trace_outline_clean_tile_index.shp"

out_dir = r"/media/ross/ssd/00_2015_DAR_marinedebris/maui/labels"

out_path_gpkg = os.path.join(out_dir, "maui_md_label_envelopes.gpkg")
out_path_pos_tiles = os.path.join(out_dir, "positive_tiles.txt")
out_path_neg_tiles = os.path.join(out_dir, "negative_tiles.txt")

In [3]:
in_pt_labels = gpd.read_file(in_labels)
in_tindex = gpd.read_file(in_tile_index)

In [4]:
in_tindex.head()

,filename,filename_1,filename_2,geometry
0,maui_10122,None,None,"POLYGON ((739498.0945460335 2315795.921187642,..."
1,maui_10192,None,None,"POLYGON ((760544.4945460336 2315795.921187642,..."
2,maui_10193,None,None,"POLYGON ((760798.0945460335 2315795.921187642,..."
3,maui_10231,None,None,"POLYGON ((772498.0945460335 2315667.481187642,..."
4,maui_10232,None,None,"POLYGON ((772627.7345460335 2315795.921187642,..."


In [5]:
#Create envelopes from the points based on the attribute 'max_size_meters'

In [6]:
def buffer_by_feat(row):
    return row.geometry.buffer(row.max_size_meters)

In [7]:
buffered = in_pt_labels.copy()

In [8]:
buffered['geometry'] = buffered.apply(buffer_by_feat, axis=1)

In [9]:
envelopes= buffered.copy()

In [10]:
envelopes['geometry'] = buffered.envelope
envelopes.head()

,unique_pt_id,island,segment,pt_id,lat,long,type,size,min_size_meters,max_size_meters,comment,observer,geometry
0,MA-001-0001,MA,1,1,21.022123,-156.625768,O,1,0.01,0.5,very small,AR,"POLYGON ((746767.042812153 2326430.389631396, ..."
1,MA-001-0002,MA,1,2,21.022217,-156.624061,B,1,0.01,0.5,None,AR,"POLYGON ((746944.3821923686 2326443.440532876,..."
2,MA-001-0003,MA,1,3,21.023182,-156.620965,P,1,0.01,0.5,None,AR,"POLYGON ((747264.7125466345 2326555.103654591,..."
3,MA-001-0004,MA,1,4,21.023173,-156.620963,P,1,0.01,0.5,None,AR,"POLYGON ((747264.9353615083 2326554.11005451, ..."
4,MA-001-0005,MA,1,5,21.023832,-156.619177,W,1,0.01,0.5,None,AR,"POLYGON ((747449.5547359405 2326629.859620453,..."


In [12]:
envelopes.to_file(out_path_gpkg, driver="GPKG")

In [11]:
if in_tindex.crs == envelopes.crs:
    print(f'{in_tindex.crs} and {envelopes.crs} match')
else:
    print(f'{in_tindex.crs} and {envelopes.crs} DO NOT MATCH')

{'init': 'epsg:26904'} and {'init': 'epsg:26904'} match


In [ ]:
#Find all the intersections of our envelopes and the tile index.

In [13]:
intersection = gpd.overlay(envelopes, in_tindex)

In [14]:
intersection.head()

,unique_pt_id,island,segment,pt_id,lat,long,type,size,min_size_meters,max_size_meters,comment,observer,filename,filename_1,filename_2,geometry
0,MA-001-0001,MA,1,1,21.022123,-156.625768,O,1,0.01,0.5,very small,AR,maui_1038,None,None,"POLYGON ((746767.042812153 2326430.389631396, ..."
1,MA-001-0002,MA,1,2,21.022217,-156.624061,B,1,0.01,0.5,None,AR,maui_1038,None,None,"POLYGON ((746944.3821923686 2326443.440532876,..."
2,MA-001-0003,MA,1,3,21.023182,-156.620965,P,1,0.01,0.5,None,AR,maui_1039,None,None,"POLYGON ((747264.7125466345 2326555.103654591,..."
3,MA-001-0004,MA,1,4,21.023173,-156.620963,P,1,0.01,0.5,None,AR,maui_1039,None,None,"POLYGON ((747264.9353615083 2326554.11005451, ..."
4,MA-001-0005,MA,1,5,21.023832,-156.619177,W,1,0.01,0.5,None,AR,maui_787,None,None,"POLYGON ((747449.5547359405 2326629.859620453,..."


In [22]:
all_tiles = in_tindex.filename.unique()
print(len(all_tiles))

1578


In [67]:
pos_tiles = intersection.filename.unique().astype('str')
print(len(positive_tiles))

329


In [77]:
pos_list = pos_tiles.tolist()
pos_list_ext = []

for pos in pos_list:
    new_pos = str(pos) + str('.tif')
    
    pos_list_ext.append(new_pos)
    
if len(pos_list_ext) == len(pos_tiles):
    print('positive lists match')
else:
    print('WARNING, INPUT AND OUTPUT POSTITIVE LIST LENGTHS DONT MATCH')

positive lists match


In [78]:
neg_tiles = np.setdiff1d(all_tiles, pos_tiles)
len(neg_tiles)

1249

In [80]:
neg_list = neg_tiles.tolist()
neg_list_ext = []

for neg in neg_list:
    new_neg = str(neg) + str('.tif')
    
    neg_list_ext.append(new_neg)
    
if len(neg_list_ext) == len(neg_tiles):
    print('positive lists match')
else:
    print('WARNING, INPUT AND OUTPUT POSTITIVE LIST LENGTHS DONT MATCH')

positive lists match


In [83]:
if len(pos_tiles) + len(neg_tiles) == len(all_tiles):
    print("Gottem.")
    
    with open(out_path_pos_tiles, 'w') as f:
        for item in pos_list_ext:
            f.write("%s\n" % item)

    with open(out_path_neg_tiles, 'w') as f:
        for item in neg_list_ext:
            f.write("%s\n" % item)
else:
    print(f'I think I lost some tiles: {len(pos_tiles)} pos/ {len(neg_tiles)} neg/ {len(all_tiles)} total')

Gottem.
